# Polarity Ranking Algorithm

## Import Libraries

In [1]:
import numpy as np
import pandas as pd

from snorkel.labeling import labeling_function

## Load Dataset

The Pandas DataFrame will consist of the following fields: 
* **Source**

In [2]:
## Load Pandas dataset


## Labeling Functions

In [1]:
# Define the label mappings for convenience
ABSTAIN = -1
LEFT = 0
CENTER = 1
RIGHT = 2

In [3]:
@labeling_function()
def lf_source_eval(x): 
    if x.bias == 'hyper_left':
        return LEFT 
    
@labeling_function()
def lf_contains_left_keywords(x): 
    # todo: if the article contains a substantial amount
    # of these phrases, then classify as left, else, center
    return ABSTAIN

@labeling_function()
def lf_contains_right_keywords(x): 
    # todo: if the article contains a substantial amount
    # of these phrases, then classify as left
    return ABSTAIN

## Deep Learning Model